In [2]:
import os

In [15]:
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
from sklearn.feature_selection import mutual_info_classif
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import math

In [16]:
training_set = pd.read_csv('data/train.csv')
features=training_set

In [17]:
for column in features:
    if features[column].dtype == 'object':
        one_hot = pd.get_dummies(features[column]).astype(int)
        one_hot.columns = [f'{col}_{column}' for col in one_hot.columns]

        features = features.drop(columns=column)
        features = features.join(one_hot, lsuffix='_df1', rsuffix='_df2')

In [18]:
features = features.dropna()
sales_price = features['SalePrice']
features = features.drop(columns='SalePrice')

In [19]:
#mutual_info = mutual_info_classif(features, sales_price)

In [20]:
top_features_indices = np.argsort(mutual_info)[::-1][:15]
selected_features = features.columns[top_features_indices]

NameError: name 'mutual_info' is not defined

In [43]:
# after first run through, these are the solidifed columns from the top feature select, kept here to 
# avoid rerunning the method on 288 columns

solidified_features = ['AllPub_Utilities', 'Pave_Street', 'CompShg_RoofMatl', 
                        'Gtl_LandSlope', 'Norm_Condition2', 'TA_GarageCond', 
                        'Y_CentralAir', 'SBrkr_Electrical', 'KitchenAbvGr', 
                        'Y_PavedDrive', 'Typ_Functional', 'TA_GarageQual', 
                        'GasA_Heating', 'TA_BsmtCond', 'WD_SaleType']

features = features[solidified_features]

In [44]:
seed = 5
np.random.seed(seed)
tf.set_random_seed(seed)
train_index = np.random.choice(len(features), round(len(features) * 0.7), replace=False)

In [45]:
X_train = features.head(int(len(features) * 0.7))

y_train = sales_price.head(int(len(features) * 0.7))

X_test = features.tail(int(len(features) * 0.3))

y_test = sales_price.tail(int(len(features) * 0.3))

In [46]:
# determining information gain for each feature and isolating most important features


In [47]:
def z_normalization(feat_val):
    feat_mean = np.mean(feat_val, axis=0)
    feat_sd = np.std(feat_val, axis=0)
    # Perform normalization
    return (feat_val - feat_mean) / feat_sd

In [53]:
#z normalize
# X_train = z_normalization(X_train)
# X_test = z_normalization(X_test)

In [60]:
X_train.shape

(784, 15)

In [79]:
# Define optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=8, activation='linear', input_shape=(15,)),
    tf.keras.layers.Dense(units=4, activation='linear'),
    tf.keras.layers.Dense(units=1, activation='linear')])
# Compile the model
model.compile(optimizer=optimizer, loss='mean_squared_error')

In [80]:
numpy_x_train = X_train.values
numpy_y_train = y_train.values

In [81]:
model.fit(numpy_x_train, numpy_y_train, epochs=200)

Epoch 1/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 943us/step - loss: 42494529536.0000 
Epoch 2/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 892us/step - loss: 42493669376.0000
Epoch 3/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 844us/step - loss: 42492674048.0000
Epoch 4/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 860us/step - loss: 42491432960.0000
Epoch 5/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 820us/step - loss: 42489802752.0000
Epoch 6/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 792us/step - loss: 42487595008.0000
Epoch 7/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 833us/step - loss: 42484625408.0000
Epoch 8/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 917us/step - loss: 42480713728.0000
Epoch 9/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 807us/step - loss: 42475696128.0000
Epoch 10/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 885us/step - loss: 42469416960.0000
Epoch 11/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 894us/step - loss: 42461728768.0000
Epoch 12/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 997us/step - loss: 42452484096.0000
Epoch 13/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 932us/step - loss

In [77]:
numpy_x_test = X_test.values
numpy_y_test = y_test.values

In [85]:
results = model.predict(numpy_x_test)

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [88]:
differences = []
for i in range(len(results)):
    differences.append(results[i] - numpy_y_test[i])    

In [91]:
np.std(differences)

77569.80130546285